In [23]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict

In [24]:
collection_name = 'week7_cora_full_dropout_ensemble'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 12


In [25]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [26]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [27]:
experiments[0]['result'].keys()

dict_keys(['val_loss-val-train-labels-0', 'val_accuracy-val-train-labels-0', 'val_loss-val-reduced-0', 'val_accuracy-val-reduced-0', 'auroc_gpc-full:24-isomap', 'aucpr_gpc-full:24-isomap', 'auroc_7-mog:24-isomap', 'aucpr_7-mog:24-isomap', 'auroc_gpc-full:28-isomap', 'aucpr_gpc-full:28-isomap', 'auroc_7-mog:28-isomap', 'aucpr_7-mog:28-isomap', 'auroc_gpc-full:24-isomap_no-edges', 'aucpr_gpc-full:24-isomap_no-edges', 'auroc_7-mog:24-isomap_no-edges', 'aucpr_7-mog:24-isomap_no-edges', 'auroc_gpc-full:28-isomap_no-edges', 'aucpr_gpc-full:28-isomap_no-edges', 'auroc_7-mog:28-isomap_no-edges', 'aucpr_7-mog:28-isomap_no-edges', 'auroc_gpc-full:32-isomap_no-edges', 'aucpr_gpc-full:32-isomap_no-edges', 'auroc_7-mog:32-isomap_no-edges', 'aucpr_7-mog:32-isomap_no-edges', 'auroc_total-predictive-entropy', 'aucpr_total-predictive-entropy', 'auroc_expected-softmax-entropy', 'aucpr_expected-softmax-entropy', 'auroc_mutual-information', 'aucpr_mutual-information', 'auroc_logit-energy', 'aucpr_logit-en

In [28]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    if any('Operating_Systems' in label for label in experiment['config']['data']['val_labels']):
        ood = 'os'
    else:
        ood = 'ai'
    
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        med = np.median(np.array(v))
        if 'auroc' in k:
            if 'gpc' in k or 'mog' in k: continue
            if 'no-edges' in k:
                no_edges = True
                k = k.replace('_no-edges', '')
            else:
                no_edges = False
            if 'softmax_entropy' in k:
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'softmax entropy', '', 'mean')] =  [mean]
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'softmax entropy', '', 'std')] =  [std]
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'softmax entropy', '', 'median')] =  [med]
                
            else:
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'feature density', k.replace('auroc:', ''), 'mean')] = [mean]
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'feature density', k.replace('auroc:', ''), 'std')] = [std]
                d[(ood, remove_other, residual, spectral_norm, no_edges, 'feature density', k.replace('auroc:', ''), 'median')] = [med]
        elif 'val_accuracy' in k:
            k = k.replace('val_accuracy', 'acc')
            d[(ood, remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            d[(ood, remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
            d[(ood, remove_other, residual, spectral_norm, '', k, '', 'median')] = [med]
                
df = pd.DataFrame(d).T.sort_index()

In [29]:
df

0
ai False True False False feature density        auroc_expected-softmax-entropy mean    0.94
                                                                                median  0.95
                                                                                std     0.04
                                                 auroc_logit-energy             mean    0.93
                                                                                median  0.94
...                                                                                      ...
os True  True True        acc-val-reduced-0                                     median  0.60
                                                                                std     0.04
                          acc-val-train-labels-0                                mean    0.59
                                                                                median  0.59
                                                                                std     0.04

[216 rows x 1 columns]

In [30]:
df = df.reset_index((0, 1, 2, 3, -1))

In [31]:
df

level_0  level_1  \
False feature density        auroc_expected-softmax-entropy      ai    False   
                             auroc_expected-softmax-entropy      ai    False   
                             auroc_expected-softmax-entropy      ai    False   
                             auroc_logit-energy                  ai    False   
                             auroc_logit-energy                  ai    False   
...                                                             ...      ...   
      acc-val-reduced-0                                          os     True   
                                                                 os     True   
      acc-val-train-labels-0                                     os     True   
                                                                 os     True   
                                                                 os     True   

                                                             level_2  level_3  \
False feature density        auroc_expected-softmax-entropy     True    False   
                             auroc_expected-softmax-entropy     True    False   
                             auroc_expected-softmax-entropy     True    False   
                             auroc_logit-energy                 True    False   
                             auroc_logit-energy                 True    False   
...                                                              ...      ...   
      acc-val-reduced-0                                         True     True   
                                                                True     True   
      acc-val-train-labels-0                                    True     True   
                                                                True     True   
                                                                True     True   

                                                            level_7     0  
False feature density        auroc_expected-softmax-entropy    mean  0.94  
                             auroc_expected-softmax-entropy  median  0.95  
                             auroc_expected-softmax-entropy     std  0.04  
                             auroc_logit-energy                mean  0.93  
                             auroc_logit-energy              median  0.94  
...                                                             ...   ...  
      acc-val-reduced-0                                      median  0.60  
                                                                std  0.04  
      acc-val-train-labels-0                                   mean  0.59  
                                                             median  0.59  
                                                                std  0.04  

[216 rows x 6 columns]

In [32]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_3', 'level_7']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [33]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('OOD', 'Remove OOD', 'Residual', 'Spectral Norm', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('Remove-Edges', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T
df_cat

OOD                                                                   ai  \
Remove OOD                                                         False   
Residual                                                           True    
Spectral Norm                                                      False   
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.94   
                                    auroc_logit-energy              0.93   
                                    auroc_mutual-information        0.65   
                                    auroc_total-predictive-entropy  0.93   
             acc-val-reduced-0                                      0.72   
             acc-val-train-labels-0                                 0.73   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                                                               median   
Remove-Edges                                                                
False        feature density        auroc_expected-softmax-entropy   0.95   
                                    auroc_logit-energy               0.94   
                                    auroc_mutual-information         0.66   
                                    auroc_total-predictive-entropy   0.94   
             acc-val-reduced-0                                       0.72   
             acc-val-train-labels-0                                  0.72   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                              
Stat                                                                 std   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.04   
                                    auroc_logit-energy              0.03   
                                    auroc_mutual-information        0.06   
                                    auroc_total-predictive-entropy  0.05   
             acc-val-reduced-0                                      0.04   
             acc-val-train-labels-0                                 0.04   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                      True    
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.90   
                                    auroc_logit-energy              0.79   
                                    auroc_mutual-information        0.31   
                                    auroc_total-predictive-entropy  0.92   
             acc-val-reduced-0                                      0.58   
             acc-val-train-labels-0                                 0.59   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                        

In [34]:
df_cat.iloc[:, :4 * 3]

OOD                                                                   ai  \
Remove OOD                                                         False   
Residual                                                           True    
Spectral Norm                                                      False   
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.94   
                                    auroc_logit-energy              0.93   
                                    auroc_mutual-information        0.65   
                                    auroc_total-predictive-entropy  0.93   
             acc-val-reduced-0                                      0.72   
             acc-val-train-labels-0                                 0.73   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                                                               median   
Remove-Edges                                                                
False        feature density        auroc_expected-softmax-entropy   0.95   
                                    auroc_logit-energy               0.94   
                                    auroc_mutual-information         0.66   
                                    auroc_total-predictive-entropy   0.94   
             acc-val-reduced-0                                       0.72   
             acc-val-train-labels-0                                  0.72   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                              
Stat                                                                 std   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.04   
                                    auroc_logit-energy              0.03   
                                    auroc_mutual-information        0.06   
                                    auroc_total-predictive-entropy  0.05   
             acc-val-reduced-0                                      0.04   
             acc-val-train-labels-0                                 0.04   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                      True    
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.90   
                                    auroc_logit-energy              0.79   
                                    auroc_mutual-information        0.31   
                                    auroc_total-predictive-entropy  0.92   
             acc-val-reduced-0                                      0.58   
             acc-val-train-labels-0                                 0.59   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                        

In [35]:
df_cat.iloc[:, 12:24]

OOD                                                                   ai  \
Remove OOD                                                         True    
Residual                                                            True   
Spectral Norm                                                      False   
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.93   
                                    auroc_logit-energy              0.92   
                                    auroc_mutual-information        0.67   
                                    auroc_total-predictive-entropy  0.92   
             acc-val-reduced-0                                      0.75   
             acc-val-train-labels-0                                 0.74   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                                                               median   
Remove-Edges                                                                
False        feature density        auroc_expected-softmax-entropy   0.94   
                                    auroc_logit-energy               0.92   
                                    auroc_mutual-information         0.69   
                                    auroc_total-predictive-entropy   0.93   
             acc-val-reduced-0                                       0.75   
             acc-val-train-labels-0                                  0.74   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                              
Stat                                                                 std   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.02   
                                    auroc_logit-energy              0.03   
                                    auroc_mutual-information        0.07   
                                    auroc_total-predictive-entropy  0.03   
             acc-val-reduced-0                                      0.03   
             acc-val-train-labels-0                                 0.04   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                      True    
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.87   
                                    auroc_logit-energy              0.79   
                                    auroc_mutual-information        0.32   
                                    auroc_total-predictive-entropy  0.90   
             acc-val-reduced-0                                      0.60   
             acc-val-train-labels-0                                 0.59   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                        

In [36]:
df_cat.iloc[:, 24:36]

OOD                                                                   os  \
Remove OOD                                                          True   
Residual                                                           False   
Spectral Norm                                                      False   
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.95   
                                    auroc_logit-energy              0.94   
                                    auroc_mutual-information        0.61   
                                    auroc_total-predictive-entropy  0.93   
             acc-val-reduced-0                                      0.78   
             acc-val-train-labels-0                                 0.78   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                                                               median   
Remove-Edges                                                                
False        feature density        auroc_expected-softmax-entropy   0.95   
                                    auroc_logit-energy               0.94   
                                    auroc_mutual-information         0.61   
                                    auroc_total-predictive-entropy   0.93   
             acc-val-reduced-0                                       0.77   
             acc-val-train-labels-0                                  0.77   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                              
Stat                                                                 std   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.02   
                                    auroc_logit-energy              0.03   
                                    auroc_mutual-information        0.06   
                                    auroc_total-predictive-entropy  0.02   
             acc-val-reduced-0                                      0.03   
             acc-val-train-labels-0                                 0.03   

OOD                                                                       \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                                      True    
Stat                                                                mean   
Remove-Edges                                                               
False        feature density        auroc_expected-softmax-entropy  0.89   
                                    auroc_logit-energy              0.85   
                                    auroc_mutual-information        0.32   
                                    auroc_total-predictive-entropy  0.93   
             acc-val-reduced-0                                      0.72   
             acc-val-train-labels-0                                 0.70   

OOD                                                                        \
Remove OOD                                                                  
Residual                                                                    
Spectral Norm                                                               
Stat                        

In [37]:
df_cat.iloc[:, 36:48]

Empty DataFrame
Columns: []
Index: [(False, feature density, auroc_expected-softmax-entropy), (False, feature density, auroc_logit-energy), (False, feature density, auroc_mutual-information), (False, feature density, auroc_total-predictive-entropy), (, acc-val-reduced-0, ), (, acc-val-train-labels-0, )]